In [1]:
!pip install yfinance

import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import numpy as np

In [13]:
ticker_input = widgets.Text(description="Ticker:")
capital_input = widgets.FloatSlider(description="Capital ($):", min=1000, max=100000, step=10)
win_prob_input = widgets.FloatSlider(description="Win Prob (p):", min=0.1, max=0.99, step=0.01)
risk_reward_input = widgets.FloatSlider(description="Risk-Reward (b):", min=0.1, max=5, step=0.1)
kelly_fraction_input = widgets.FloatSlider(description="Kelly Fraction:", min=0.1, max=1, step=0.1)
fetch_data_checkbox = widgets.Checkbox(description="Auto-calculate p & b from historical data?", value=False)
calculate_button = widgets.Button(description="Calculate Kelly Position")
output = widgets.Output()

input_box = widgets.VBox([
    widgets.HBox([ticker_input, fetch_data_checkbox]),
    capital_input,
    win_prob_input,
    risk_reward_input,
    kelly_fraction_input,
    calculate_button
])

def kelly_criterion(p, b, fraction=1):
    q = 1 - p
    f_star = (b * p - q) / b
    return max(0, f_star) * fraction

def get_historical_stats(ticker):
    data = yf.download(ticker, period="1y")["Close"]
    returns = data.pct_change().dropna()
    positive_returns = returns[returns > 0]
    p = len(positive_returns) / len(returns)
    avg_gain = positive_returns.mean()
    avg_loss = returns[returns <= 0].mean()
    b = abs(avg_gain / avg_loss)
    return p, b

def on_calculate_button_click(b):
    with output:
        output.clear_output()

        capital = capital_input.value
        fraction = kelly_fraction_input.value

        if fetch_data_checkbox.value:
            try:
                p, b = get_historical_stats(ticker_input.value)
                if hasattr(p, 'item'): p = p.item()
                if hasattr(b, 'item'): b = b.item()
                win_prob_input.value = p
                risk_reward_input.value = b
            except Exception as e:
                print(f"Error fetching data: {e}. Using manual inputs.")
                p, b = win_prob_input.value, risk_reward_input.value
        else:
            p, b = win_prob_input.value, risk_reward_input.value

        optimal_f = kelly_criterion(p, b, fraction)
        position_size = optimal_f * capital

        print(f"Optimal Position Size: ${position_size:.2f}")
        print(f"Kelly Fraction: {optimal_f * 100:.1f}% of capital")
        print(f"Win Probability (p): {p:.2f}")
        print(f"Risk-Reward Ratio (b): {b:.2f}")

calculate_button.on_click(on_calculate_button_click)

display(input_box, output)

Output()